# Source Location Tracking

Learn how to track where prompts are defined in your source code for better debugging and provenance.

In [1]:
from t_prompts import prompt

## What is Source Location Tracking?

Source location tracking captures **where in your source code** each prompt was created. This is different from source mapping (which tracks where text comes from in the rendered output).

By default, every prompt automatically captures:
- **Filename** and **filepath** - which file the prompt was created in
- **Line number** - the line in the file where the prompt was created

This information is stored in a `SourceLocation` object attached to every element (both static text and interpolations).

The implementation is lightweight - it only uses information directly available from Python's stack frames without reading any source files.

## Basic Usage

Source location is captured automatically - no configuration needed!

In [2]:
task = "translate to French"
p = prompt(t"Task: {task}")

# Access source location from an interpolation
loc = p['task'].source_location

print("This prompt was created at:")
print(f"  File: {loc.filename}")
print(f"  Line: {loc.line}")
print(f"\nFormatted: {loc.format_location()}")

This prompt was created at:
  File: 1658211292.py
  Line: 2

Formatted: 1658211292.py:2


## Checking Availability

Source location might not be available in some contexts (REPL, eval, etc.). Use the `is_available` property to check:

In [3]:
loc = p['task'].source_location

if loc.is_available:
    print(f"Location: {loc.format_location()}")
else:
    print("Source location not available")

Location: 1658211292.py:2


## Source Location for All Elements

Both **static text** and **interpolations** have source location information:

In [4]:
from t_prompts import Static, StructuredInterpolation

name = "Alice"
p = prompt(t"Hello {name}!")

print("All elements with their source locations:\n")
for elem in p.elements:
    loc = elem.source_location
    if isinstance(elem, Static):
        print(f"Static text: {elem.value!r}")
    elif isinstance(elem, StructuredInterpolation):
        print(f"Interpolation '{elem.key}': {elem.value!r}")

    if loc and loc.is_available:
        print(f"  → Created at {loc.format_location()}")
    print()

All elements with their source locations:

Static text: 'Hello '
  → Created at 4066850082.py:4

Interpolation 'name': 'Alice'
  → Created at 4066850082.py:4

Static text: '!'
  → Created at 4066850082.py:4



## Nested Prompts Have Separate Locations

Each prompt (including nested ones) captures its own creation location:

In [5]:
# Create inner prompt on one line
greeting = "Hello"
inner = prompt(t"{greeting}, world!")

# Create outer prompt on another line
outer = prompt(t"Message: {inner:msg}")

# They have different source locations
outer_loc = outer['msg'].source_location
inner_loc = inner['greeting'].source_location

print(f"Outer prompt created at line {outer_loc.line}")
print(f"Inner prompt created at line {inner_loc.line}")
print(f"\nThey're on different lines: {outer_loc.line != inner_loc.line}")

Outer prompt created at line 6
Inner prompt created at line 3

They're on different lines: True


## Exporting Source Location in Provenance

Source location is automatically included when you export provenance:

In [6]:
task = "analyze sentiment"
p = prompt(t"Task: {task}")

prov = p.to_provenance()

# Source location is in the provenance
node = prov['nodes'][0]
print("Provenance for interpolation 'task':")
print(f"  Key: {node['key']}")
print(f"  Expression: {node['expression']}")
print(f"  Source location: {node.get('source_location', 'not available')}")

Provenance for interpolation 'task':
  Key: task
  Expression: task
  Source location: {'filename': '2810673975.py', 'filepath': '/private/var/folders/wr/5n6v6bvj6mxdxjcg5_5ss1ph0000gn/T/ipykernel_98578/2810673975.py', 'line': 2}


## Performance: Disabling Source Location

Source location capture uses Python's `inspect` module to walk the stack. While lightweight (no file I/O), you can disable it if needed:

In [7]:
task = "translate"
p = prompt(t"Task: {task}", capture_source_location=False)

# Source location is None when disabled
print(f"Source location: {p['task'].source_location}")

Source location: None


## Use Case: Debugging Complex Prompt Hierarchies

When building large prompt systems with many nested prompts, source location helps you quickly find where each piece was defined:

In [8]:
def create_system_prompt():
    role = "helpful assistant"
    return prompt(t"You are a {role}.")

def create_user_message(task):
    return prompt(t"Task: {task}")

def build_conversation(task):
    system = create_system_prompt()
    user = create_user_message(task)
    return prompt(t"System: {system:sys}\nUser: {user:usr}")

# Build the conversation
conv = build_conversation("translate to French")

# Trace where each part was created
print("Prompt component origins:\n")
for key in ['sys', 'usr']:
    loc = conv[key].source_location
    if loc and loc.is_available:
        print(f"{key}: {loc.filename}:{loc.line}")

Prompt component origins:

sys: 1883045048.py:11
usr: 1883045048.py:11


## Use Case: Audit Logging

Track which code paths generated specific prompts for compliance and debugging:

In [9]:
def log_prompt_creation(p, name):
    """Log where a prompt was created."""
    # Get location from first interpolation (if any)
    if p.interpolations:
        loc = p.interpolations[0].source_location
        if loc and loc.is_available:
            print(f"[AUDIT] Prompt '{name}' created at {loc.filepath}:{loc.line}")
        else:
            print(f"[AUDIT] Prompt '{name}' created (location unavailable)")

# Example usage
sensitive_data = "user@example.com"
p = prompt(t"Email: {sensitive_data:email}")
log_prompt_creation(p, "user_email_prompt")

[AUDIT] Prompt 'user_email_prompt' created at /private/var/folders/wr/5n6v6bvj6mxdxjcg5_5ss1ph0000gn/T/ipykernel_98578/973597698.py:13


## SourceLocation API Reference

The `SourceLocation` dataclass has the following fields:

- **`filename`**: Short filename (e.g., `'script.py'`)
- **`filepath`**: Full absolute path to the file
- **`line`**: Line number where prompt was created (1-indexed)

All fields are optional (may be `None` if source is unavailable).

### Methods

- **`is_available`**: Property that returns `True` if location info is present
- **`format_location()`**: Returns a formatted string like `"script.py:42"`

## Summary

Source location tracking provides automatic provenance for your prompts:

✅ **Enabled by default** - no configuration needed  
✅ **Lightweight** - uses only stack frame info, no file I/O  
✅ **Works everywhere** - static text, interpolations, and nested prompts  
✅ **Exported in provenance** - included in `to_provenance()` output  
✅ **Opt-out available** - disable for performance with `capture_source_location=False`  

This makes debugging complex prompt systems much easier!